# US Visas

In [1]:
#Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cleanup

# Input which data files are available in the
from subprocess import check_output

cleanup.generate_cleaned_df()

C:\Users\andre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0,1,2,3,4,5,6,7,10,11,16,17,20,21,22,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,41,42,43,44,45,47,48,49,50,51,52,53,55,56,57,58,59,60,61,63,64,65,66,68,69,70,71,72,73,74,77,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,100,101,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,153) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\andre\PycharmProjects\python_visa\cleanup.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

   wage_offer_from wage_offer_unit_of_pay case_received_date decision_date  \
0         75629.00                     yr                NaT    2012-02-01   
1         37024.00                     yr                NaT    2011-12-21   
2         47923.00                     yr                NaT    2011-12-01   
3            10.97                     hr                NaT    2011-12-01   
4        100000.00                     yr                NaT    2012-01-26   

  employer_state    case_number foreign_worker_info_education_other  \
0       NEW YORK  A-07323-97014                                 NaN   
1       NEW YORK  A-07332-99439                                 NaN   
2       VIRGINIA  A-07333-99643                                 NaN   
3       NEW YORK  A-07339-01930                                 NaN   
4       NEW YORK  A-07345-03565                                 NaN   

  country_of_citizenship case_status     us_economic_sector  \
0                ARMENIA   Certified     